In [1]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()

# Load the API key from an environment variable
CORE_API_KEY = os.getenv('CORE_API_KEY')
BASE_URL = 'https://api.core.ac.uk/v3'

def fetch_data(endpoint, params=None):
    """
    Fetches data from the specified CORE API endpoint.
    
    Args:
        endpoint (str): The endpoint of the API to call.
        params (dict): Optional dictionary of parameters to include in the request.
        
    Returns:
        dict: The JSON response from the API as a dictionary.
    """
    headers = {
        'Authorization': f'Bearer {CORE_API_KEY}'
    }
    response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        # Raise an error if the request was not successful
        response.raise_for_status()

def get_single_research(query):
    """
    Retrieves a single research article with selected metadata fields.
    
    Args:
        query (str): The search query (e.g., "machine learning").
        
    Returns:
        dict: Selected metadata fields of the first research article that matches the search query.
    """
    search_params = {
        'q': query,
        'page': 1,
        'pageSize': 1
    }
    
    # Search for articles
    search_results = fetch_data('search/works', params=search_params)
    
    # Extract the first result's ID
    if 'results' in search_results and len(search_results['results']) > 0:
        article_id = search_results['results'][0]['id']
        
        # Fetch full metadata and text for the article
        article_data = fetch_data(f'works/{article_id}')
        
        # Extract specific fields
        title = article_data.get('title', 'No title available')
        authors = [author.get('name', 'Unknown') for author in article_data.get('authors', [])]
        year = article_data.get('year', 'No year available')
        abstract = article_data.get('abstract', 'No abstract available')
        full_text = article_data.get('fullText', 'No full text available')
        full_text_link = article_data.get('downloadUrl', 'No link available')
        
        # Additional metadata fields
        keywords = article_data.get('keywords', 'No keywords available')
        publisher = article_data.get('publisher', 'No publisher available')
        publication_date = article_data.get('publicationDate', 'No publication date available')
        journal = article_data.get('journal', {}).get('name', 'No journal available')
        citation_count = article_data.get('citationCount', 'No citation count available')
        
        # Format and return selected metadata fields
        return {
            "Title": title,
            "Authors": ", ".join(authors),
            "Year": year,
            "Abstract": abstract,
            "Keywords": keywords,
            "Publisher": publisher,
            "Publication Date": publication_date,
            "Journal": journal,
            "Citation Count": citation_count,
            "Full Text": full_text,  # Full text included without truncation
            "Link": full_text_link
        }
    else:
        print("No results found for the query.")
        return None

def save_metadata_to_file(data, filename):
    """
    Saves the article metadata to a JSON file for RAG model use.
    
    Args:
        data (dict): The article metadata.
        filename (str): The name of the file to save the data.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Data saved to {filename}")

# Example usage of the function
if __name__ == "__main__":
    query = "Large Language Model"  # Change the query as needed
    article = get_single_research(query)
    
    if article:
        filename = "article_metadata.json"
        save_metadata_to_file(article, filename)

Data saved to article_metadata.json


In [2]:
from langchain_openai import OpenAIEmbeddings

import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=OPENAI_API_KEY
    # dimensions=1024  # Uncomment if you need to specify dimensions
)

In [3]:
with open("article_metadata.json", "r", encoding="utf-8") as file:
    article_metadata = json.load(file)

text_content = (
    f"Title: {article_metadata.get('Title', 'No title available')}\n"
    f"Authors: {article_metadata.get('Authors', 'Unknown')}\n"
    f"Year: {article_metadata.get('Year', 'No year available')}\n"
    f"Abstract: {article_metadata.get('Abstract', 'No abstract available')}\n"
    f"Full Text: {article_metadata.get('Full Text', 'No full text available')}\n"
    f"Link: {article_metadata.get('Link', 'No link available')}\n"
)

# Get the embedded vector
embedded_vector = embeddings.embed_query(text_content)

# Display the embedded vector
print("Embedded Vector:", embedded_vector)

Embedded Vector: [-0.022033881722982346, 0.0004073067154089401, -0.024232240503552764, 0.0006018093542234152, 0.009674012721916906, 0.006412234568082427, -0.01668626991042513, 0.03532986606184318, -0.02981754514079396, 0.007397652615168719, 0.028799619863622276, 0.021586983400190603, 0.015195470808446493, -0.0449417677373098, -0.0009340629368175323, 0.0018062717644852816, 0.01865953860276772, 0.008246372393755657, 0.010717214697301832, -0.04031715124953352, -0.02329554154078393, -0.015632261564107405, -0.029142924624469605, 0.037472403109069026, -0.016138463586945227, 0.023011156398517627, 0.003992652078585584, 0.002940928846824014, -0.04836636744629095, 0.012514732198243946, 0.019388018850591795, -0.0029536167089380416, 0.0037292994121164913, -0.009283063023777823, -0.027389171281014892, -0.031244334103869734, 0.08412456377494582, 0.028965310843761544, -0.026267499670231467, 0.05685823527179413, 0.030183632052093956, 0.01846354811310829, -0.00865744534513138, -0.027765091300548817, 0.

In [4]:
import json
file_path = "article_metadata.json"

with open(file_path, "r", encoding="utf-8") as file:
    article_metadata = json.load(file)

In [5]:
import json
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import lancedb

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize the embeddings model
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY
)

# List of file paths for JSON files
file_paths = ["article_metadata.json", "article_metadata copy.json"]

# Initialize a list to hold all embedded data
embedded_vectors = []

# Process each JSON file
for file_path in file_paths:
    # Load the article metadata from JSON
    with open(file_path, "r", encoding="utf-8") as file:
        article_metadata = json.load(file)

    # Embed the full text
    embedding = embeddings_model.embed_query(article_metadata["Full Text"])

    # Append the structured data to embedded_vectors
    embedded_vectors.append({
        "embedding": embedding,
        "metadata": {
            "Title": article_metadata["Title"],
            "Authors": article_metadata["Authors"],
            "Year": article_metadata["Year"],
            "Abstract": article_metadata["Abstract"],
            "Keywords": article_metadata["Keywords"],
            "Link": article_metadata["Link"]
        }
    })

# Connect to LanceDB and create the table with all data
db = lancedb.connect("./lancedb")
tbl = db.create_table("my_table", data=embedded_vectors)

print("Data from both JSON files successfully stored in LanceDB.")


c:\Hackathon\HackUMass-XII\HackUMass-XII\HackUMass-XII\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data from both JSON files successfully stored in LanceDB.


In [6]:
import numpy as np
import pandas as pd
from langchain_openai import OpenAIEmbeddings
import lancedb

# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Initialize the embeddings model
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY
)

# Connect to LanceDB (local database)
db = lancedb.connect("./lancedb")
table_name = "my_table"
tbl = db.open_table(table_name)

# Define the query text
query_text = "Large margin criterion for training neural language models."

# Embed the query text
query_embedding = embeddings_model.embed_query(query_text)

# Convert LanceDB table to a Pandas DataFrame
df = tbl.to_pandas()

# Calculate similarity between query and each embedding in the DataFrame
similarities = []
for _, row in df.iterrows():
    embedding = row["embedding"]
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((row, similarity))

# Sort by similarity in descending order and get the top 5 results
similarities.sort(key=lambda x: x[1], reverse=True)
top_results = similarities[:5]

# Display the results
for result, similarity in top_results:
    print("Title:", result['metadata']['Title'])
    print("Authors:", result['metadata']['Authors'])
    print("Abstract:", result['metadata']['Abstract'])
    print("Similarity Score:", similarity)
    print("-" * 50)


Title: Large Margin Neural Language Model
Authors: Huang, Jiaji, Huang, Liang, Li, Yi, Ping, Wei
Abstract: We propose a large margin criterion for training neural language models.
Conventionally, neural language models are trained by minimizing perplexity
(PPL) on grammatical sentences. However, we demonstrate that PPL may not be the
best metric to optimize in some tasks, and further propose a large margin
formulation. The proposed method aims to enlarge the margin between the "good"
and "bad" sentences in a task-specific sense. It is trained end-to-end and can
be widely applied to tasks that involve re-scoring of generated text. Compared
with minimum-PPL training, our method gains up to 1.1 WER reduction for speech
recognition and 1.0 BLEU increase for machine translation.Comment: 9 pages. Accepted as a long paper in EMNLP201
Similarity Score: 0.6161682362747671
--------------------------------------------------
Title: Large Margin Neural Language Model
Authors: Huang, Jiaji, Huang, L

In [6]:
import json
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Load environment variables from .env
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# Load the full text from article_metadata.json
file_path = "article_metadata.json"
with open(file_path, "r", encoding="utf-8") as file:
    article_metadata = json.load(file)

# Extract the full text
full_text = article_metadata.get("Full Text", "")

# Define the first prompt for a concise summary
summary_prompt = ChatPromptTemplate.from_messages(
    [("system", "Summarize the paper concisely:\n\n{context}")]
)

# Define the second prompt for suggestions to other researchers
suggestions_prompt = ChatPromptTemplate.from_messages(
    [("system", "Based on this paper, suggest potential directions or recommendations for other researchers in the same field:\n\n{context}")]
)

# Set up chains with the prompt, llm, and output parser
summary_chain = summary_prompt | llm | StrOutputParser()
suggestions_chain = suggestions_prompt | llm | StrOutputParser()

# Invoke the chains with the full text as context
summary_result = summary_chain.invoke({"context": full_text})
suggestions_result = suggestions_chain.invoke({"context": full_text})

# Print the results
print("Concise Summary of the Article:")
print(summary_result)
print("\nSuggestions for Other Researchers in the Same Field:")
print(suggestions_result)


Concise Summary of the Article:
The paper introduces a large margin criterion for training neural language models (NLMs), contrasting traditional methods that minimize perplexity (PPL). It argues that PPL does not always yield optimal results for tasks like automatic speech recognition (ASR) and machine translation (MT). The proposed large margin formulation aims to enhance the discrimination between "good" and "bad" sentences in a task-specific context. Two models are proposed: the Large Margin Language Model (LMLM), which focuses on comparing references with imperfect hypotheses, and the ranking-loss-based LMLM (rLMLM), which ranks all hypotheses. Experiments demonstrate that both LMLM and rLMLM outperform conventional PPL-based NLMs in ASR and MT tasks, achieving significant reductions in word error rate (WER) and increases in BLEU scores, despite higher PPL values. The findings suggest that optimizing for task-specific metrics leads to better performance in practical applications o